## Anaconda Env Setting

In [ ]:
conda create -n some_env_name
conda install python=3.10 numpy pytorch torchvision pandas scikit-learn jupyter notebook

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
import pandas as pd

## Download Resources

In [ ]:
# CIFAR-10
CIFAR_transform_train = transforms.Compose([transforms.ToTensor()])
CIFAR_transform_test =transforms.Compose([transforms.ToTensor()])

trainset_CIFAR = datasets.CIFAR10(root='./data', train=True, download=True, transform= CIFAR_transform_train)
testset_CIFAR = datasets.CIFAR10(root='./data', train=False, download=True, transform=CIFAR_transform_test)
CIFAR_train = DataLoader(trainset_CIFAR, batch_size=32, shuffle=True, num_workers=2)
CIFAR_test = DataLoader(testset_CIFAR, batch_size=32, shuffle=False, num_workers=2)
CIFAR_train_images = []
CIFAR_train_labels = []
for batch in CIFAR_train:
  images, labels = batch
  images_flat = images.view(images.shape[0], -1)
  CIFAR_train_images.append(images_flat.numpy())
  CIFAR_train_labels.append(labels.numpy())
CIFAR_train_images = np.vstack(CIFAR_train_images)
CIFAR_train_labels = np.concatenate(CIFAR_train_labels)

CIFAR_test_images = []
CIFAR_test_labels = []
for batch in CIFAR_test:
  images, labels = batch
  images_flat = images.view(images.shape[0], -1)
  CIFAR_test_images.append(images_flat.numpy())
  CIFAR_test_labels.append(labels.numpy())
CIFAR_test_images = np.vstack(CIFAR_test_images)
CIFAR_test_labels = np.concatenate(CIFAR_test_labels)

models = {}

## Decision Tree Test

In [ ]:
model = DecisionTreeClassifier(max_depth=3)
model.fit(CIFAR_train_images, CIFAR_train_labels)
score = model.score(CIFAR_test_images, CIFAR_test_labels)
print("Model %d score : %f\nParameters : %s" % (model.max_depth, score, model.get_params()))

## Decision Tree Assignment

In [ ]:
from sklearn.model_selection import cross_validate
params_grid = {
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4], 
  'max_leaf_nodes': [5, 10, None]
}

grid_searchs=[]
result=[]
for depth in [3,6,9,12]:
    model = DecisionTreeClassifier(max_depth=depth)
    grid_search = GridSearchCV(model, param_grid=params_grid, cv=5)
    grid_search.fit(CIFAR_train_images, CIFAR_train_labels)
    
    accuracy_training = grid_search.score(CIFAR_train_images, CIFAR_train_labels)
    accuracy_test = grid_search.score(CIFAR_test_images, CIFAR_test_labels)
    result.append([depth, accuracy_training, accuracy_test])
    grid_searchs.append(grid_search)
    print('depth %d best score : %f' % (depth, grid_search.best_score_))

pd.DataFrame(result, columns=["Depth", "Accuracy(Training Set)", "Accuracy(Test Set)"])

## SVM Test

## SVM Assignment